# Project predicting redox potential values 

data HOMO LUMO energies, fingerprints different combinations of them

# load SHAP tool 

In [ ]:
pip install shap

#  load tools 

We can load various ML tools in python, in this project most od them come from sklearn
SHAP is not in sklearn but you can load it with pip install commend (above)

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
import shap
# from rdkit import Chem
# from rdkit.Chem.Draw import rdMolDraw2D
from sklearn.pipeline import Pipeline
print(' load ')

In [ ]:
!dir  Re*

# Data 

Redox-dataset.csv  : data with descriptors and target (rodoxpot1, 1st redoxpotential)

HomoA : HOMO of molec A 
LumoA : LUMO of molec A 
GapA  : HOMO-LUMO gap of molec A 

HomoAH	:  HOMO of molc AH
LumoAH	:  LUMO of molec AH
GapAH	:  gap of molec AH 

HomoAH2	:  same for molc AH2 
LumoAH2	
GapAH2	
SIMILES : similes of the molecules

finger256.csv : fingerprints (256 different values) of the molecules  

In [ ]:
data_all = pd.read_csv("Redox-dataset.csv")

finger_all = pd.read_csv("Redox-finger256.csv")   # 256 fingerp

data=data_all.head(2000)       # first 2000, this is more than the data so full data set is used 
finger=finger_all.head(2000)   # with large data sets it is usefull to use only part of the data for testing 
                               # and then go the full data

print('full data:', data_all.shape)
print('working data:', data.shape)

# show the data (in python first and last few lines)

data

# create three different datasets:  

finger + HOMO-LUMO data

finger only 

Homo-Lumo only

In [ ]:
# finger
redpot=data['redoxpot1']

# combine the fingerprints and the other data 
X2=pd.concat([finger,data],axis=1)   # FP's first

# fingerprint and target 
Xf=pd.concat([finger,redpot],axis=1)   # only FP and target 

# remove ID redpot2 and SMILES from the data 

X2=X2.drop(columns= ['ID','redoxpot2','SMILES'])

# only the data 
Xd=data.drop(columns= ['ID','redoxpot2','SMILES'])
Target='redoxpot1'

In [ ]:
X2.columns

In [ ]:
Xf.columns

In [ ]:
Xd.columns

In [ ]:
# do the data split for the 3 data sets. Here 25% to the test set 

y_all = X2[Target]
train_data, test_data = train_test_split(X2, test_size=0.25)
train_fdata, test_fdata = train_test_split(Xf, test_size=0.25)
train_ddata, test_ddata = train_test_split(Xd, test_size=0.25)

In [ ]:
print('train_data.size:',train_data.shape )
print('test_data.size:',test_data.shape )
print('train_fdata.size:',train_fdata.shape )
print('test_fdata.size:',test_fdata.shape )
print('train_ddata.size:',train_ddata.shape )
print('test_ddata.size:',test_ddata.shape )
#testd

In [ ]:
print(train_data.head)   # note that the data order has been randomized. This is needed because the original data 
print(train_fdata.head)  # is usually stored in some prefefined order taht may bias the ML 

# Descriptor  Finger and other  (X contain the descriptors and y is the target)

In [ ]:
X = train_data.drop(columns= [Target]) 
y = train_data[Target]
###
test_X = test_data.drop(columns= [Target]) 
#
test_y = test_data[Target]
###

# Descriptor  Finger only

In [ ]:
Xf = train_fdata.drop(columns= [Target]) # .to_numpy()
yf = train_fdata[Target]
###
test_Xf = test_fdata.drop(columns= [Target]) # .to_numpy()
#unscale_tX = test_X
test_yf = test_fdata[Target]
###
print(Xf)


In [ ]:
Xd = train_ddata.drop(columns= [Target]) # .to_numpy()
yd = train_ddata[Target]
###
test_Xd = test_ddata.drop(columns= [Target]) # .to_numpy()
#unscale_tX = test_X
test_yd = test_ddata[Target]
###
print(Xd)

# choose GB, KR or RF the code will do 3 data sets

# Which of these methods predict the data best ?

In [ ]:


ML="GB"

if ML=="GB":
#  # GB method next 2 lines
  model = GradientBoostingRegressor()
  parameters = {'learning_rate': np.arange(0.05, 0.3, 0.05), "loss": ['huber'],"n_estimators": range(20, 80, 10),
                'subsample': [1.0, 0.9]}   # for GB

# learning_rate, loss,  n_estimators and subsample  are thechnical parameters of the GB ML methods. We will find 
# the best combination of them for this data. This is quite technical and you need to look the manual page of the ML method


if ML=="KR":
  model = KernelRidge()
  parameters = {"alpha": np.arange(0.5, 3.5, 0.2), "kernel": ['linear', 'polynomial', 'laplacian'], "degree": [2,3]}  #  KRR

if ML=="RF":
# RF methods next 2 lines
  model = RandomForestRegressor()
  parameters = {"n_estimators": range(110, 190, 20), "min_samples_split":[2,3]}  # for RF

if ML=="GP":
# GaussianProc methods next 2 lines. NOTE this will do the fitting here and there is no parameter optimization
  kernel = DotProduct() + WhiteKernel()
  model = GaussianProcessRegressor(kernel=kernel,random_state=0).fit(X,y)

                                                                      
# Grid serach will find the best parametetrs cv is the number of cross validation cycles (min cv = 5, 10 is better but slower)
###
if ML!="GP":
  clf = GridSearchCV(model, parameters, cv = 10, verbose=2, n_jobs=-1, scoring='neg_mean_squared_error')
  clff = GridSearchCV(model, parameters, cv = 10, verbose=2, n_jobs=-1, scoring='neg_mean_squared_error')
  clfd = GridSearchCV(model, parameters, cv = 10, verbose=2, n_jobs=-1, scoring='neg_mean_squared_error')

# Train the ML method for this data 

In [ ]:
from time import time

# fit the finger + data desc
time0=time()
if ML!="GP":
  clf.fit(X,y)
timee=time()
print('time used',timee-time0,' s')

In [ ]:
# fit the finger only desc

time0=time()
if ML!="GP":
  clff.fit(Xf,yf)
timee=time()
print('time used',timee-time0,' s')

In [ ]:
# fit the data only desc
time0=time()
if ML!="GP":
  clfd.fit(Xd,yd)
timee=time()
print('time used',timee-time0,' s')

print the best parameters

In [ ]:
print(clf.best_params_)

print(clff.best_params_)

print(clfd.best_params_)

fix the models

In [ ]:
model = clf.best_estimator_
print(model)
modelf = clff.best_estimator_
print(modelf)
modeld = clfd.best_estimator_
print(modeld)

The few best models are not very sensitve to the parameters

In [ ]:
result_df = pd.DataFrame(clf.cv_results_)
result_df = result_df.sort_values("rank_test_score")
for i, row in result_df.iterrows():
   if(i<10): print(row["mean_test_score"], row["params"])

# SHAP analysis

In [ ]:

#  for train data 
print('===== SHAP FOR TRAIN DATA ====')
print('===== NOTE SHAP ANALYSIS DOES NOT WORK WITH KR METHOD ====')
smodel=modeld.fit(Xd,yd)      # rather slow
explainer = shap.Explainer(smodel)
shap_values = explainer(Xd)
print('===== SHAP DONE ====')

In [ ]:
features = Xd.columns

featureNames=[]
for i in range(0,len(features)-1): # ignore the first and last feature 
    featureNames.append(features[i])
nFeatures=len(featureNames)
print('Feature Names ', featureNames)

Xs = Xd  # data2   # this have the headers SHAP
print('\n shap base_values ',shap_values.base_values[0])
base=shap_values.base_values[0]
shap.plots.beeswarm(shap_values,max_display=15) #
shap.plots.bar(shap_values) # ,feature_names=featureNames) # all shap values

#shap.plots.waterfall(shap_values[11])


In [ ]:
print(' =========    SHAP 2 ANALYSIS START ========')
shap2_values = shap.TreeExplainer(smodel).shap_values(Xd)  # this is slow  X for train data, X2 for all 
print(' =========    SHAP 2 ANALYSIS DONE ========')

In [ ]:

F1="HomoAH2"
F2="LumoA"
F3="GapAH"
F4="GapA"
print(' =========    SHAP DEPENDENCE  =============')
# shap.dependence_plot(F1, shap2_values, Xd)
shap.dependence_plot(F2, shap2_values, Xd)
#shap.dependence_plot(F3, shap2_values, Xd)
#shap.dependence_plot(F4, shap2_values, Xd)

# Prediction analysis. Find  which data set is the best descriptor


# FULL fingerprint + data 

In [ ]:
pred_y = model.predict(X)
#print(' shape y',np.shape(y),'shape X',np.shape(X))
yy=[]   # y is suffled  yy contain the unsuffled data
i=0
for yp in y:
#  print('test y  ',i,yp)
  yy.append(yp)
  i+=1
print(' shape y',np.shape(y),'shape X',np.shape(X))

cut=0.25
print('\n------------Test large deviations --(cut >',cut,')-------------')
for ii in range(len(y)):
    if abs(pred_y[ii]-yy[ii]) > cut:
      print('Pred E ',pred_y[ii],' E_DFT',yy[ii],'diff',abs(pred_y[ii]-yy[ii]))

print('---------------------------------------------------------------')

plt.plot(pred_y, y, 'o', label = "ML")
plt.plot(y, y)
plt.xlabel("Predicted Energy (eV)")
plt.ylabel("DFT energy (eV)")
plt.legend()
r_sqrt = r2_score(y, pred_y)
e_mse = mse(y,pred_y)  # Mean squared error 
e_mae = mae(y,pred_y)  # Mean abs error 
print("\n R^2 on ML training set:", r_sqrt)
print(" MSE on ML training set:", e_mse)
print(" MAE on ML training set:", e_mae)

# you can save the r2, MSE and MAE values 

if(ML=="GB"):
    GB_r2_full=r_sqrt
    GB_MSE_full=e_mse
    GB_MAE_full=e_mae
    
if(ML=="KR"):
    KR_r2_full=r_sqrt
    KR_MSE_full=e_mse
    KR_MAE_full=e_mae

if(ML=="RF"):
    RF_r2_full=r_sqrt
    RF_MSE_full=e_mse
    RF_MAE_full=e_mae

# Finager print only

In [ ]:
print(' FINGERPRINT ONLY \n')
pred_yf = modelf.predict(Xf)
#print(' shape y',np.shape(y),'shape X',np.shape(X))
yy=[]   # y is suffled  yy contain the unsuffled data
i=0
for yp in yf:
#  print('test y  ',i,yp)
  yy.append(yp)
  i+=1
#print('y(0:18) \n',y[0:8],yy[0:8])
print(' shape y',np.shape(y),'shape X',np.shape(X))

cut=0.25
print('\n------------Test large deviations --(cut >',cut,')-------------')
for ii in range(len(y)):
    if abs(pred_yf[ii]-yy[ii]) > cut:
#      print('Xii, i',ii,X[ii,:])
#       Xtest=X[ii,:].reshape(1, -1) 
#       model.predict(Xtest)
      print('Pred E ',pred_yf[ii],' E_DFT',yy[ii],'diff',abs(pred_yf[ii]-yy[ii]))

print('---------------------------------------------------------------')


plt.plot(pred_yf, yf, 'o', label = "ML")
plt.plot(yf, yf)
plt.xlabel("Predicted Energy (eV)")
plt.ylabel("DFT energy (eV)")
plt.legend()
r_sqrt = r2_score(yf, pred_yf)
e_mse = mse(yf,pred_yf)  # Mean squared error 
e_mae = mae(yf,pred_yf)  # Mean abs error 
print("\n R^2 on ML training set:", r_sqrt)
print(" MSE on ML training set:", e_mse)
print(" MAE on ML training set:", e_mae)


In [ ]:
print(' HOMO LUMO DATA ONLY \n')
pred_yd = modeld.predict(Xd)
#print(' shape y',np.shape(y),'shape X',np.shape(X))
yy=[]   # y is suffled  yy contain the unsuffled data
i=0
for yp in yd:
#  print('test y  ',i,yp)
  yy.append(yp)
  i+=1
# print('y(0:18) \n',y[0:8],yy[0:8])
print(' shape y',np.shape(y),'shape X',np.shape(X))

cut=0.25
print('\n------------Test large deviations --(cut >',cut,')-------------')
for ii in range(len(y)):
    if abs(pred_yd[ii]-yy[ii]) > cut:
      print('Pred E ',pred_yf[ii],' E_DFT',yy[ii],'diff',abs(pred_yf[ii]-yy[ii]))

print('---------------------------------------------------------------')


plt.plot(pred_yd, yd, 'o', label = "ML")
plt.plot(yd, yd)
plt.xlabel("Predicted Energy (eV)")
plt.ylabel("DFT energy (eV)")
plt.legend()
r_sqrt = r2_score(yd, pred_yd)
e_mse = mse(yd,pred_yd)  # Mean squared error 
e_mae = mae(yd,pred_yd)  # Mean abs error 
print("\n R^2 on ML training set:", r_sqrt)
print(" MSE on ML training set:", e_mse)
print(" MAE on ML training set:", e_mae)


In [ ]:
#test_X = test_data.drop(columns= ['dEf', 'name']).to_numpy()
#test_y = test_data['dEf']
print(' FULL data Test set \n') 
print(' shape y',np.shape(test_y),'shape X',np.shape(test_X))
predt_y = model.predict(test_X)

yy=[]   # test_y is suffled  yy is the unsuffled data
i=0
for yp in test_y:
#  print('test y  ',i,yp)
  yy.append(yp)
  i+=1

cut=0.3
print('\n------------Test large deviations --(cut >',cut,')-------------')
for ii in range(len(test_y)):
    if abs(predt_y[ii]-yy[ii]) > cut:
#      print('test_X, i',ii,test_X[ii,:])
#       Xtest=X[ii,:].reshape(1, -1) 
      print('Pred E ',predt_y[ii],' E_DFT',yy[ii],'diff',abs(predt_y[ii]-yy[ii]))
print('-----------------------------------------------------------------')
r_sqrt = r2_score(test_y, predt_y)
e_mse = mse(test_y,predt_y)  # Mean squared error 
e_mae = mae(test_y,predt_y)  # Mean abs error 
print("\n R^2 on ML TEST set:", r_sqrt)
print(" MSE on ML TEST set:", e_mse)
print(" MAE on ML TEST set:", e_mae)

plt.plot(predt_y, test_y, 'o', label = "ML")
plt.plot(test_y, test_y)
plt.xlabel("Predicted redox (eV)")
plt.ylabel("DFT redox (eV)")
plt.legend()
plt.show()



In [ ]:
# Normal importance analysis

feature_importance = modeld.feature_importances_
data_used = train_ddata.drop(columns= [Target])
plt.bar(data_used.columns, feature_importance)
plt.show()

In [ ]:
print(feature_importance)